In [93]:
library(devtools)
library(ggridges)
set.seed(1)

load_all("../../../hmmTMB")

ℹ Loading hmmTMB


In [94]:
dists <- list(
  norm_obs       = "norm",
  dir_obs        = "dir",
  mvnorm_obs     = "mvnorm"
)

params <- list(
  norm_obs       = list(mean     = c( 0,  3),   sd       = c(1, 2)),
  dir_obs        = list(alpha1   = c( 2,  5),   alpha2   = c( 3, 1)),
  mvnorm_obs     = list(mu1      = c( 0,  2),   mu2      = c( 0, -2),
                        sd1      = c( 1,  2),   sd2      = c(1, 1),
                        corr12   = c( 0,  0.5))
)

evaluation_vals <- list(
  norm_obs       = seq(-3, 3, by = 0.10),
  dir_obs        = sapply(seq(0, 1, 0.05), function(x) {c(x, 1 - x)}),
  mvnorm_obs     = t(as.matrix(expand.grid(seq(-3,3,1), seq(-3,3,1))))
)


In [95]:
n_training <- 2
training_df <- data.frame(
  ID = rep(1, n_training),
  norm_obs = c(NA, NA),
  dir_obs = c(NA, NA),
  mvnorm_obs = I(replicate(n_training, c(NA, NA), simplify = FALSE))
)

tpm <- matrix(c(0.8, 0.3, 0.2, 0.7), 2, 2)
starting_state <- c(0.8, 0.2)

## Create True Model ----------------------------------------------------
hid_mod <- MarkovChain$new(
  data = training_df,
  n_states = 2,
  tpm = tpm
)
hid_mod$update_delta0(starting_state)
obs_mod <- Observation$new(
  data = training_df,
  n_states = 2,
  dists = dists,
  par = params
)
true_mod <- HMM$new(obs = obs_mod, hid = hid_mod)

In [100]:
n_forecast <- 2
forecast_df <- data.frame(
  ID = rep(1, n_forecast),
  norm_obs = c(NA, NA),
  dir_obs = c(NA, NA),
  mvnorm_obs = I(replicate(n_forecast, c(NA, NA), simplify = FALSE))
)

forecast = Forecast$new(
  hmm = true_mod,
  forecast_data = forecast_df,
  preset_x_vals = evaluation_vals,
  starting_state_distribution = starting_state,
)

In [101]:
forecast$forecasted_pdfs[['dir_obs']]

0.00000000,0.00000000
0.02115610,0.01827674
0.03797982,0.03281341
0.05083378,0.04393007
0.06008792,0.05195759
0.06611948,0.05723769
0.06931305,0.06012297
0.07006055,0.06097684
0.06876121,0.06017360
0.06582158,0.05809835
0.06165556,0.05514708


In [102]:
forecast$forecasted_pdfs[['mvnorm_obs']]

8.380064e-04,1.285085e-03
2.840555e-03,4.261598e-03
6.978735e-03,1.020514e-02
1.150599e-02,1.682544e-02
1.273049e-02,1.909916e-02
1.020901e-02,1.565554e-02
6.085613e-03,9.399533e-03
5.051557e-04,6.503366e-04
3.827217e-03,4.324699e-03
1.543204e-02,1.672174e-02
2.827952e-02,3.195544e-02
